# COMP90051 Project 2

In [54]:
# Do not edit. These are the only imports permitted.
%matplotlib inline
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

## 1. Implementing ε-Greedy and UCB

In [55]:
class MAB(ABC):
    """
    Abstract class that represents a multi-armed bandit (MAB)
    """
    
    @abstractmethod
    def play(self, tround, context):
        """
        Play a round
        
        Arguments
        =========
        tround : int
            positive integer identifying the round
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to the arms
        
        Returns
        =======
        arm : int
            the positive integer arm id for this round
        """
    
    @abstractmethod
    def update(self, arm, reward, context):
        """
        Updates the internal state of the MAB after a play
        
        Arguments
        =========
        arm : int
            a positive integer arm id in {1, ..., self.narms}
        
        reward : float
            reward received from arm
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to arms
        """

In [56]:
class EpsGreedy(MAB):
    """
    Epsilon-Greedy multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    epsilon : float
        explore probability

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, epsilon, Q0=np.inf):
        self.narms = narms
        self.epsilon = epsilon
        self.Q = np.full(narms, Q0)
        self.Q_counts = np.zeros(narms, dtype=np.int64)
        self.Q_sums = np.zeros(narms)
        
        
    def play(self, tround, context=None):
        exploit = np.argmax(np.random.random(self.Q.shape) * (self.Q == self.Q.max()))
        explore = np.random.randint(self.narms)
        
        if np.random.random_sample() <= self.epsilon:
            return explore
        else:
            return exploit
        
        
    def update(self, arm, reward, context=None):
        self.Q_sums[arm-1] += reward
        self.Q_counts[arm-1] += 1
        
        for i in range(self.narms):
            if self.Q_counts[i] > 0:
                self.Q = self.Q_sums[i] / self.Q_counts[i]
        

In [57]:
class UCB(MAB):
    """
    Upper Confidence Bound (UCB) multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    rho : float
        positive real explore-exploit parameter

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, rho, Q0=np.inf):
        self.narms = narms
        self.rho = rho
        self.Q = np.full(narms, Q0)
        self.Q_counts = np.zeros(narms, dtype=np.int64)
        self.Q_sums = np.zeros(narms)
        
    
    def play(self, tround, context=None):
        for i in range(self.narms):
            if self.Q_counts[i] > 0:
                self.Q[i] = (self.Q_sums[i] / self.Q_counts[i]) + np.sqrt((self.rho*np.log(tround)) / self.Q_sums[i])
                
        it = np.argmax(np.random.random(self.Q.shape) * (self.Q == self.Q.max()))
        return it
        
        
    def update(self, arm, reward, context=None):
        self.Q_sums[arm-1] += reward
        self.Q_counts[arm-1] += 1
        
    

## 2. The Basic Thompson Bandit

In [72]:
class BetaThompson(MAB):
    """
    Beta-Bernoulli Thompson sampling multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    alpha0 : float, optional
        positive real prior hyperparameter

    beta0 : float, optional
        positive real prior hyperparameter
    """
    
    def __init__(self, narms, alpha0=1.0, beta0=1.0):
        self.narms = narms
        self.alpha0 = alpha0
        self.beta0 = beta0
        self.S = np.full(narms, 0)
        self.F = np.full(narms, 0)
        
        
    def play(self, tround, context=None):
        theta = np.array([np.random.beta(self.S[i]+self.alpha0, self.F[i]+self.beta0) for i in range(self.narms)])
        return np.argmax(np.random.random(theta.shape) * (theta == theta.max()))
        
        
    def update(self, arm, reward, context=None):
        if reward == 1:
            self.S[arm-1] += 1
        else:
            self.F[arm-1] += 1

## 3. Off-Policy Evaluation

In [92]:
def offlineEvaluate(mab, arms, rewards, contexts, nrounds=None):
    """
    Offline evaluation of a multi-armed bandit
    
    Arguments
    =========
    mab : instance of MAB
    
    arms : 1D int array, shape (nevents,) 
        integer arm id for each event
    
    rewards : 1D float array, shape (nevents,)
        reward received for each event
    
    contexts : 2D float array, shape (nevents, mab.narms*nfeatures)
        contexts presented to the arms (stacked horizontally) 
        for each event.
        
    nrounds : int, optional
        number of matching events to evaluate `mab` on.
    
    Returns
    =======
    out : 1D float array
        rewards for the matching events
    """
    T = arms.shape[0]
    R = []
    tround = 1
    for i in range(T):
        
        arm = arms[i]
        context = contexts[i]
        reward = rewards[i]
        
        if mab.play(tround, context) == arm:
            R.append(reward)
            mab.update(arm, reward, context)
            tround += 1
            
            if nrounds is not None and tround >= nrounds:
                break
            
    return np.array(R)
    

In [93]:
data = open('dataset.txt')

In [94]:
data = []
with open('dataset.txt') as fp:
    for row in fp:
        data.append(list(map(int, row.split())))
        
data = np.array(data)
arms = data[:,0]
rewards = data[:,1]
contexts = data[:,2:]

print(data.shape)
print(arms.shape)
print(rewards.shape)
print(contexts.shape)

(10000, 102)
(10000,)
(10000,)
(10000, 100)


In [98]:
mab = EpsGreedy(10, 0.05)
results_EpsGreedy = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('EpsGreedy average reward', np.mean(results_EpsGreedy))

EpsGreedy average reward 0.117647058824


In [99]:
mab = UCB(10, 1.0)
results_UCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('UCB average reward', np.mean(results_UCB))

C:\Users\Allan\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: RuntimeWarning: divide by zero encountered in double_scalars


UCB average reward 0.0951188986233


In [100]:
mab = BetaThompson(10, 1.0, 1.0)
results_BetaThompson = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('BetaThompson average reward', np.mean(results_BetaThompson))

BetaThompson average reward 0.12140175219


C:\Users\Allan\Anaconda3\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full(10, 0) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


## 4. Contextual Bandits - LinUCB

In [ ]:
class LinUCB(MAB):
    """
    Contextual multi-armed bandit (LinUCB)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    alpha : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, alpha):
        
        
    def play(self, tround, context):
        
    
    def update(self, arm, reward, context):
        
    

In [ ]:
mab = LinUCB(10, 10, 1.0)
results_LinUCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinUCB average reward', np.mean(results_LinUCB))

## 5. Contextual Bandits - LinThompson

In [ ]:
class LinThompson(MAB):
    """
    Contextual Thompson sampled multi-armed bandit (LinThompson)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    v : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, v):
        
        
    def play(self, tround, context):
        
    
    def update(self, arm, reward, context):
        
    

In [ ]:
mab = LinThompson(10, 10, 1.0)
results_LinThompson = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinThompson average reward', np.mean(results_LinThompson))

## 6. Evaluation
### 6.A.

### 6.B.